# CDC reported inpatient beds time series

* [Estimated inpatient beds](https://healthdata.gov/sites/default/files/estimated_inpatient_all_20210221_1631.csv)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from bs4 import BeautifulSoup
import requests

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
y, m, d = datetime.now().year, datetime.now().month, datetime.now().day
datasource_site = "https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-state"
response = requests.get(datasource_site)
assert response.status_code == 200
soup = BeautifulSoup(response.content)
link = soup.find("a", {"class": "data-link"})
assert link
url = link["href"]

In [ ]:
modified_date = soup.find("div", {"property": "dcterms:modified"}).text

In [ ]:
df = pd.read_csv(url)

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]
df["ISO3166_1"] = "US"
df["ISO3166_2"] = df["state"]
df["LAST_UPDATE_DATE"] = modified_date
df["LAST_UPDATE_DATE"] = pd.to_datetime(df["LAST_UPDATE_DATE"].astype(str), format='%Y-%m-%d')
df['Last_Reported_Flag'] = df['LAST_UPDATE_DATE'].max() == df['LAST_UPDATE_DATE']
df = df[df["ISO3166_2"].isin(states)]

```sql
CREATE OR REPLACE TABLE CDC_REPORTED_PATIENT_IMPACT (
  STATE varchar(2),
  CRITICAL_STAFFING_SHORTAGE_TODAY_YES integer,
  CRITICAL_STAFFING_SHORTAGE_TODAY_NO integer,
  CRITICAL_STAFFING_SHORTAGE_TODAY_NOT_REPORTED integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_YES integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_NO integer,
  CRITICAL_STAFFING_SHORTAGE_ANTICIPATED_WITHIN_WEEK_NOT_REPORTED integer,
  HOSPITAL_ONSET_COVID integer,
  HOSPITAL_ONSET_COVID_COVERAGE integer,
  INPATIENT_BEDS integer,
  INPATIENT_BEDS_COVERAGE integer,
  INPATIENT_BEDS_USED integer, 
  INPATIENT_BEDS_USED_COVERAGE integer,
  INPATIENT_BEDS_USED_COVID integer, 
  INPATIENT_BEDS_USED_COVID_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_CONFIRMED integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_CONFIRMED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_SUSPECTED integer,
  PREVIOUS_DAY_ADMISSION_ADULT_COVID_SUSPECTED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_CONFIRMED integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_CONFIRMED_COVERAGE integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_SUSPECTED integer,
  PREVIOUS_DAY_ADMISSION_PEDIATRIC_COVID_SUSPECTED_COVERAGE integer,
  STAFFED_ADULT_ICU_BED_OCCUPANCY integer,
  STAFFED_ADULT_ICU_BED_OCCUPANCY_COVERAGE integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_AND_SUSPECTED_COVID integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_COVID integer,
  STAFFED_ICU_ADULT_PATIENTS_CONFIRMED_COVID_COVERAGE integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_COVID integer,
  TOTAL_ADULT_PATIENTS_HOSPITALIZED_CONFIRMED_COVID_COVERAGE integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_AND_SUSPECTED_COVID_COVERAGE integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_COVID integer,
  TOTAL_PEDIATRIC_PATIENTS_HOSPITALIZED_CONFIRMED_COVID_COVERAGE integer, 
  TOTAL_STAFFED_ADULT_ICU_BEDS integer,
  TOTAL_STAFFED_ADULT_ICU_BEDS_COVERAGE integer,
  INPATIENT_BEDS_UTILIZATION float,
  INPATIENT_BEDS_UTILIZATION_COVERAGE integer,
  INPATIENT_BEDS_UTILIZATION_NUMERATOR integer,
  INPATIENT_BEDS_UTILIZATION_DENOMINATOR integer, 
  PERCENT_OF_INPATIENTS_WITH_COVID float,
  PERCENT_OF_INPATIENTS_WITH_COVID_COVERAGE integer,
  PERCENT_OF_INPATIENTS_WITH_COVID_NUMERATOR integer,
  PERCENT_OF_INPATIENTS_WITH_COVID_DENOMINATOR integer,
  INPATIENT_BED_COVID_UTILIZATION float,
  INPATIENT_BED_COVID_UTILIZATION_COVERAGE integer,
  INPATIENT_BED_COVID_UTILIZATION_NUMERATOR integer,
  INPATIENT_BED_COVID_UTILIZATION_DENOMINATOR integer,
  ADULT_ICU_BED_COVID_UTILIZATION float,
  ADULT_ICU_BED_COVID_UTILIZATION_COVERAGE integer,
  ADULT_ICU_BED_COVID_UTILIZATION_NUMERATOR integer,
  ADULT_ICU_BED_COVID_UTILIZATION_DENOMINATOR integer,
  ADULT_ICU_BED_UTILIZATION float,
  ADULT_ICU_BED_UTILIZATION_COVERAGE integer,
  ADULT_ICU_BED_UTILIZATION_NUMERATOR integer,
  ADULT_ICU_BED_UTILIZATION_DENOMINATOR integer,
  REPORTING_CUTOFF_START timestamp_ntz,
  ISO3166_1 varchar(2),
  ISO3166_2 varchar(2),
  LAST_UPDATE_DATE timestamp_ntz,
  LAST_REPORTED_FLAG boolean
);
```

In [ ]:
df.to_csv(output_folder + "CDC_REPORTED_PATIENT_IMPACT.csv", index=False, columns=df.columns)